In [1]:
import xarray as xr
import os 
import glob
#import netCDF4 as nc
import numpy as np
import pandas as pd
#import geopandas as gpd
import netCDF4 as nc
import matplotlib.pyplot as plt
#import regionmask

In [2]:
def attr_vars_read(head):
    vars_list = []
    dir_subfolders = []
    for sub_folder in os.listdir(head):
        print(sub_folder)
        files = os.listdir(os.path.join(head,sub_folder))
        dir_subfolders.append(os.path.join(head,sub_folder))
        med_dir = os.path.join(head,sub_folder,files[0])
        dset = nc.Dataset(med_dir)
        med_var = list(set(dset.variables.keys()-['longitude','latitude','time']))
        if len(med_var) == 1:
            vars_list.append(med_var[0])
        else:
            print("The %s has external vars (%)" %(sub_folder,med_var))   
    sub_files = os.listdir(dir_subfolders[0])
    sample = xr.open_dataset(os.path.join(head,dir_subfolders[0],sub_files[0]))
    #sample = getattr(sample_file, vars_list[0])    
    return dir_subfolders, vars_list, sample

In [6]:
from tqdm import notebook as tqdm
import time
# import numba  
 
# @numba.jit(nopython=True) 
def data_mean(dir_folder,dir_var):
    sub_filenames = os.listdir(os.path.join(dir_folder))
    ds = xr.open_dataset(os.path.join(dir_folder,sub_filenames[0]))
#     min_lon = 114 
#     min_lat = 31.5 
#     max_lon = 119.5 
#     max_lat = 35.5
    min_lon = 111.5 
    min_lat = 30.5 
    max_lon = 121.8 
    max_lat = 36.8
    mask_lon = (ds.longitude >= min_lon) & (ds.longitude <= max_lon)
    mask_lat = (ds.latitude >= min_lat) & (ds.latitude <= max_lat)
    data = np.zeros([len(sub_filenames), 64,104])
    for i in tqdm.tqdm(range(len(sub_filenames))):
        med_ds = xr.open_dataset(os.path.join(dir_folder,sub_filenames[i]))
        ds_hbp = med_ds.where(mask_lon & mask_lat, drop=True)
        ds_aggre = ds_hbp.resample(time='1D').mean(skipna=True)
        med_data = getattr(ds_aggre, dir_var)
        data[i,:,:] = med_data[0,:,:]
    #template = getattr(sample, 'evavt')  
    #template = getattr(sample, 'swvl1')
    #data_masked = np.where(~np.isnan(MASK.region),data, np.nan)
    return data#data_masked

In [7]:
#from multiprocessing import Pool
head = r'G:\ERA5_Land_surface-HRB\soil_moisture'
#head = r'G:\ERA5_validation_period'
vars_folder, vars_attr, ds  = attr_vars_read(head)
##
# ds_var = list(set(ds.variables.keys()-['longitude','latitude','time']))
# ds_single = getattr(ds, ds_var[0])  
# mask = xr.open_dataset(r'G:\mask_info\Spain_mainland.nc') 
# outline = gpd.read_file(r'G:\mask_info\Spian_mainland\spain_mainland.shp')

Volumetric soil water layer 1-HRB


In [8]:
ii = 0
ds_m = data_mean(vars_folder[ii],vars_attr[ii])

  0%|          | 0/14980 [00:00<?, ?it/s]

In [9]:
#ds = xr.open_dataset(os.path.join(vars_folder[0],sub_filenames[0]))
min_lon = 111.5 
min_lat = 30.5 
max_lon = 121.8 
max_lat = 36.8
mask_lon = (ds.longitude >= min_lon) & (ds.longitude <= max_lon)
mask_lat = (ds.latitude >= min_lat) & (ds.latitude <= max_lat)
# save as nc
template = getattr(ds, vars_attr[0])
lat = template.latitude
lon = template.longitude
LON,LAT = np.meshgrid(lon[mask_lon],lat[mask_lat])
reference_time = pd.Timestamp("now")
date = pd.date_range(start='19791231',end='20210103',freq='D')
da = xr.Dataset(
    {vars_attr[0]: (("time","rows", "colms"), ds_m)},
    coords=dict(
        time=date,
        lon=(['rows', "colms"], LON),
        lat=(["rows", "colms"], LAT),
        reference_time=reference_time,
    ),
    attrs=dict(
        description="daily means calculated from hourly data.",
        units="refer to the variable",
    ),  
    )
#da.to_netcdf('G:\\ERA5_Land_surface-HRB\\'+vars_attr[ii]+'_HuaibeiPlain_from_1980_to_2021.nc')

In [10]:
da_week = da['swvl1'].resample(time="7D").mean(dim="time")

In [11]:
da.to_netcdf('G:\\ERA5_Land_surface-HRB\\'+vars_attr[ii]+'_HRB_from_1980_to_2021.nc')

In [12]:
da_week.to_netcdf('G:\\ERA5_Land_surface-HRB\\'+vars_attr[ii]+'_weekly_HRB_from_1980_to_2021.nc')